In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import mlflow
import lightgbm as lgb
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import Normalizer
from sklearn import datasets
from sklearn.cluster import KMeans

In [2]:
data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Functions

In [3]:
def get_title(name):
    title=''
    for i in name:
        if i==' ':
            title=''
        elif i=='.':
            return title
        else:
            title=title+i
    return 'Without title'
def Cabin_null_processing(name):
    if (type(name) is int):
        return 0
    else:
        return 1
def Age_null_processing1(Age):
    return 37
def Age_null_processing2(Age):
    return 29
def Age_null_processing3(Age):
    return 27

## Null values processing

In [4]:
#Embarked
data['Embarked']=data['Embarked'].replace(np.nan, 'S')
test_data['Embarked']=test_data['Embarked'].replace(np.nan, 'S')

#Cabin
data['Cabin']=data['Cabin'].replace(np.nan, 0)
data['Cabin']=data['Cabin'].map(Cabin_null_processing)
data['Cabin']=data['Cabin'].replace(np.nan, 0)
data['Cabin']=data['Cabin'].map(Cabin_null_processing)
test_data['Cabin']=test_data['Cabin'].replace(np.nan, 0)
test_data['Cabin']=test_data['Cabin'].map(Cabin_null_processing)
test_data['Cabin']=test_data['Cabin'].replace(np.nan, 0)
test_data['Cabin']=test_data['Cabin'].map(Cabin_null_processing)
#data = data.drop(columns='Cabin')
#test_data = test_data.drop(columns='Cabin')
#Age
for i in  range(len(data)):
    if (pd.isna(data.loc[i, 'Age'])==True):
        if (data.loc[i,'Pclass']==1):
            data.loc[i,'Age'] = 37
        elif (data.loc[i,'Pclass']==2):
            data.loc[i,'Age'] = 29
        elif (data.loc[i,'Pclass']==3):
            data.loc[i,'Age'] = 27
for i in  range(len(test_data)):
    if (pd.isna(test_data.loc[i, 'Age'])==True):
        if (test_data.loc[i,'Pclass']==1):
            test_data.loc[i,'Age'] = 37
        elif (test_data.loc[i,'Pclass']==2):
            test_data.loc[i,'Age'] = 29
        elif (test_data.loc[i,'Pclass']==3):
            test_data.loc[i,'Age'] = 27            
#Fare
test_data['Fare']=test_data['Fare'].replace(np.nan, 20)

## Data processing

#### label encoding

In [5]:
#Sex
data['Sex']=LabelEncoder().fit_transform(data.Sex)
test_data['Sex']=LabelEncoder().fit_transform(test_data.Sex)

#Ticket
#data=data.drop(columns=['Ticket'])
#test_data=test_data.drop(columns=['Ticket'])
data['Ticket']=LabelEncoder().fit_transform(data.Ticket)
test_data['Ticket']=LabelEncoder().fit_transform(test_data.Ticket)

#New feature 'Title' & drop 'Name'
data['Title']=data['Name'].map(get_title)
data=data.drop(columns=['Name'])
test_data['Title']=test_data['Name'].map(get_title)
test_data=test_data.drop(columns=['Name'])

#Embarked
data['Embarked']=LabelEncoder().fit_transform(data.Embarked)
test_data['Embarked']=LabelEncoder().fit_transform(test_data.Embarked)

#PassengerId
ids = test_data['PassengerId']
data=data.drop(columns='PassengerId')
test_data=test_data.drop(columns='PassengerId')

#Title
data['Title']=LabelEncoder().fit_transform(data.Title)
test_data['Title']=LabelEncoder().fit_transform(test_data.Title)

In [6]:
labels = data['Survived']
data = data.drop(columns='Survived')

In [7]:
test_data.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,3,1,34.5,0,0,152,7.8292,0,1,5
1,3,0,47.0,1,0,221,7.0000,0,2,6
2,2,1,62.0,0,0,73,9.6875,0,1,5
3,3,1,27.0,0,0,147,8.6625,0,2,5
4,3,0,22.0,1,1,138,12.2875,0,2,6


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    int32  
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Ticket    891 non-null    int32  
 6   Fare      891 non-null    float64
 7   Cabin     891 non-null    int64  
 8   Embarked  891 non-null    int32  
 9   Title     891 non-null    int32  
dtypes: float64(2), int32(4), int64(4)
memory usage: 55.8 KB


In [9]:
cv_acc1=[]

# Clustering (Age, Fare)

#### All features (78.6 % kaggle public dataset)

In [10]:
model = KMeans(n_clusters = 15, random_state = 42)
#data.Age.to_numpy().shape
model.fit(data)
data['cluster']=model.labels_
test_data['cluster']=model.predict(test_data)
data.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,cluster1
0,3,1,22.0,1,0,523,7.2500,0,2,12,1
1,1,0,38.0,1,0,596,71.2833,0,0,13,5
2,3,0,26.0,0,0,669,7.9250,0,2,9,4
3,1,0,35.0,1,0,49,53.1000,0,2,13,3
4,3,1,35.0,0,0,472,8.0500,0,2,12,11


#### only Pclass and fare

In [11]:
model = KMeans(n_clusters = 3, random_state = 42)
#data.Age.to_numpy().shape
model.fit(data[['Pclass', 'Fare']])
data['cluster']=model.labels_
test_data['cluster']=model.predict(test_data[['Pclass', 'Fare']])
data.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,cluster1,cluster2
0,3,1,22.0,1,0,523,7.2500,0,2,12,1,1
1,1,0,38.0,1,0,596,71.2833,0,0,13,5,0
2,3,0,26.0,0,0,669,7.9250,0,2,9,4,1
3,1,0,35.0,1,0,49,53.1000,0,2,13,3,0
4,3,1,35.0,0,0,472,8.0500,0,2,12,11,1


#### age, sex, title

In [12]:
model = KMeans(n_clusters = 5, random_state = 42)
#data.Age.to_numpy().shape
model.fit(data[['Age', 'Sex', 'Title']])
data['cluster']=model.labels_
test_data['cluster']=model.predict(test_data[['Age', 'Sex', 'Title']])
data.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,cluster1,cluster2,cluster3
0,3,1,22.0,1,0,523,7.2500,0,2,12,1,1,1
1,1,0,38.0,1,0,596,71.2833,0,0,13,5,0,0
2,3,0,26.0,0,0,669,7.9250,0,2,9,4,1,4
3,1,0,35.0,1,0,49,53.1000,0,2,13,3,0,0
4,3,1,35.0,0,0,472,8.0500,0,2,12,11,1,0


# Random Forest

In [13]:
params = {'bootstrap': False,
 'max_depth': 70,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 600}
model = RandomForestClassifier(**params, random_state=42)
#print('cv accuracy: ', cross_val_score(model, data, labels, scoring='accuracy', cv = 10).mean())
cv_acc1.append(cross_val_score(model, data, labels, scoring='accuracy', cv = 10).mean())
model.fit(data, labels)
predictions_RF = model.predict(test_data)

# XGB

In [14]:
params={'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.05,
 'max_depth': 10,
 'min_child_weight': 0.5,
 'n_estimators': 50,
 'reg_lambda': 0.1,
 'subsample': 0.8}
model = XGBClassifier(**params , random_state=42)
#print('cv accuracy: ', cross_val_score(model, data, labels, scoring='accuracy', cv = 10).mean())
cv_acc1.append(cross_val_score(model, data, labels, scoring='accuracy', cv = 10).mean())
model.fit(data, labels)
predictions_XGB = model.predict(test_data)

D:\anaconda\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:17:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:17:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:17:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:17:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

# Catboost

In [15]:
params ={'depth': 6, 'iterations': 20, 'learning_rate': 0.2}
model = CatBoostClassifier(**params, random_state=42)
#print('cv accuracy: ', cross_val_score(model, data, labels, scoring='accuracy', cv = 10).mean())
cv_acc1.append(cross_val_score(model, data, labels, scoring='accuracy', cv = 10).mean())
model.fit(data, labels)
predictions_Catboost = model.predict(test_data)

0:	learn: 0.6254031	total: 50.4ms	remaining: 958ms
1:	learn: 0.5730977	total: 51.5ms	remaining: 463ms
2:	learn: 0.5351297	total: 52.4ms	remaining: 297ms
3:	learn: 0.5035277	total: 53.6ms	remaining: 214ms
4:	learn: 0.4780083	total: 54.9ms	remaining: 165ms
5:	learn: 0.4585356	total: 55.9ms	remaining: 130ms
6:	learn: 0.4459912	total: 56.9ms	remaining: 106ms
7:	learn: 0.4377230	total: 57.9ms	remaining: 86.8ms
8:	learn: 0.4293194	total: 58.9ms	remaining: 72ms
9:	learn: 0.4239909	total: 59.5ms	remaining: 59.5ms
10:	learn: 0.4179515	total: 60.5ms	remaining: 49.5ms
11:	learn: 0.4115662	total: 61.6ms	remaining: 41.1ms
12:	learn: 0.4053976	total: 62.6ms	remaining: 33.7ms
13:	learn: 0.4000196	total: 63.6ms	remaining: 27.3ms
14:	learn: 0.3955706	total: 64.6ms	remaining: 21.5ms
15:	learn: 0.3912443	total: 65.5ms	remaining: 16.4ms
16:	learn: 0.3880442	total: 66.2ms	remaining: 11.7ms
17:	learn: 0.3875885	total: 66.6ms	remaining: 7.4ms
18:	learn: 0.3843630	total: 67.9ms	remaining: 3.57ms
19:	learn: 0.

1:	learn: 0.5849272	total: 2.7ms	remaining: 24.3ms
2:	learn: 0.5435206	total: 3.86ms	remaining: 21.9ms
3:	learn: 0.5090611	total: 4.93ms	remaining: 19.7ms
4:	learn: 0.4848277	total: 5.98ms	remaining: 17.9ms
5:	learn: 0.4680484	total: 7.07ms	remaining: 16.5ms
6:	learn: 0.4550503	total: 8.2ms	remaining: 15.2ms
7:	learn: 0.4473263	total: 8.68ms	remaining: 13ms
8:	learn: 0.4387294	total: 10ms	remaining: 12.3ms
9:	learn: 0.4300869	total: 11.1ms	remaining: 11.1ms
10:	learn: 0.4231453	total: 12.2ms	remaining: 9.97ms
11:	learn: 0.4152116	total: 13.2ms	remaining: 8.82ms
12:	learn: 0.4106591	total: 14.2ms	remaining: 7.66ms
13:	learn: 0.4068197	total: 15.2ms	remaining: 6.54ms
14:	learn: 0.4040443	total: 17.3ms	remaining: 5.75ms
15:	learn: 0.3991551	total: 18.8ms	remaining: 4.71ms
16:	learn: 0.3944054	total: 20.1ms	remaining: 3.54ms
17:	learn: 0.3908828	total: 21.2ms	remaining: 2.35ms
18:	learn: 0.3882628	total: 22.2ms	remaining: 1.17ms
19:	learn: 0.3849329	total: 23.5ms	remaining: 0us
0:	learn: 0

# LightGBM

In [16]:
params = {'max_depth': 3, 'min_data_in_leaf': 100, 'num_leaves': 3}
model = LGBMClassifier(**params,  random_state=42)
#print('cv accuracy: ', cross_val_score(model, data, labels, scoring='accuracy', cv = 10).mean())
cv_acc1.append(cross_val_score(model, data, labels, scoring='accuracy', cv = 10).mean())
model.fit(data, labels)
predictions_LGBM = model.predict(data)


[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

# SVM

In [17]:
params = {'C': 5, 'coef0': 0.001, 'degree': 3, 'gamma': 0.0006, 'kernel': 'rbf'}
model = SVC(**params,  random_state=42)
clf = make_pipeline(StandardScaler(), model)
#print('cv accuracy: ', cross_val_score(clf, data, labels, scoring='accuracy', cv = 10).mean())
cv_acc1.append(cross_val_score(clf, data, labels, scoring='accuracy', cv = 10).mean())
clf.fit(data, labels)
predictions_SVM = clf.predict(test_data)

# Logistic Regression

In [18]:
params={'C': 1.0, 'penalty': 'l2'}
model = LogisticRegression(**params)
clf = make_pipeline(StandardScaler(), model)
#print('cv accuracy: ', cross_val_score(clf, data, labels, scoring='accuracy', cv = 10).mean())
cv_acc1.append(cross_val_score(clf, data, labels, scoring='accuracy', cv = 10).mean())
clf.fit(data, labels)
predictions_LogisticRegression = model.predict(test_data)

# kNN

In [19]:
params={'n_neighbors': 13}
model = KNeighborsClassifier(**params)
clf = make_pipeline(StandardScaler(), model)
#print('cv accuracy: ', cross_val_score(clf, data, labels, scoring='accuracy', cv = 10).mean())
cv_acc1.append(cross_val_score(clf, data, labels, scoring='accuracy', cv = 10).mean())
clf.fit(data, labels)

predictions_kNN = clf.predict(test_data)

# Final predictions

In [20]:
prediction = [0 for i in range(len(test_data))]
print(len(prediction))
for i in range(len(test_data)):
    sum_predictions=predictions_XGB[i]+predictions_SVM[i]+predictions_kNN[i]+predictions_LogisticRegression[i]+predictions_Catboost[i]+predictions_LGBM[i]
    if sum_predictions>3:
        prediction[i]=1
    else:
        prediction[i]=0

print(prediction)

output = pd.DataFrame({'PassengerId': ids, 'Survived': prediction})
output.to_csv('my_submission.csv', index=False)

418
[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,

# Encoding table

In [23]:
cv_acc1=([0.82493134, 0.81033708, 0.81705368, 0.79575531, 0.78672909,
       0.79575531, 0.81707865])


In [24]:
cv_acc2 = [0.8249313358302123, 0.8126092384519351, 0.8237952559300874, 0.7957553058676654, 0.786729088639201,
 0.7946317103620475, 0.8170911360799001]

In [25]:
cv_acc3 = [0.826067415730337, 0.8170786516853932, 0.8114856429463172, 0.7900998751560551, 0.8193133583021224,
 0.8294132334581772, 0.8361548064918851]

In [26]:
models = ['RF', 'XGB', 'CatBoost', 'LGBM', 'SVM', 'Logistic Regression', 'kNN']
zeros = [0 for i in range(7)]
df = pd.DataFrame({'model': models, 'label': cv_acc1, 'one_hot': cv_acc2, 'target':cv_acc3})

In [27]:
df

,model,label,one_hot,target
0,RF,0.824931,0.824931,0.826067
1,XGB,0.810337,0.812609,0.817079
2,CatBoost,0.817054,0.823795,0.811486
3,LGBM,0.795755,0.795755,0.790100
4,SVM,0.786729,0.786729,0.819313
5,Logistic Regression,0.795755,0.794632,0.829413
6,kNN,0.817079,0.817091,0.836155


# Clustering table

In [22]:
cv_acc1=[0.8238701622971284, 0.8361922596754058, 0.8271535580524345,0.804756554307116,0.786729088639201,
         0.7935205992509363,0.8103370786516854]

In [23]:
cv_acc2=[0.8406741573033706, 0.8473907615480648, 0.8170536828963796, 0.8047440699126094, 0.786729088639201,
 0.8013483146067415, 0.8171036204744071]

In [24]:
cv_acc3=[0.8372908863920099, 0.8417852684144819, 0.8114481897627964, 0.8036329588014981, 0.786729088639201,
 0.8024843945068664, 0.821585518102372]

In [25]:
cv_acc4 = [0.8395380774032459, 0.8417852684144819, 0.8215605493133582, 0.8058676654182273, 0.786729088639201,
           0.8036079900124845, 0.8193882646691636]

In [26]:
cv_acc5 = [0.8372908863920101, 0.8462546816479402, 0.8137078651685392, 0.8047565543071162, 0.786729088639201,
           0.7935081148564295, 0.8272159800249689]

In [27]:
models = ['RF', 'XGB', 'CatBoost', 'LGBM', 'SVM', 'Logistic Regression', 'kNN']
zeros = [0 for i in range(7)]
df = pd.DataFrame({'model': models, 'kmean5, all features': cv_acc1, 'kmean5, pclass and fare': cv_acc2,
                   'kmean5, age, sex, title':cv_acc3, 'kmean3, all features': cv_acc4, 'kmean15, all features': cv_acc5})

In [28]:
df

,model,"kmean5, all features","kmean5, pclass and fare","kmean5, age, sex, title","kmean3, all features","kmean15, all features"
0,RF,0.823870,0.840674,0.837291,0.839538,0.837291
1,XGB,0.836192,0.847391,0.841785,0.841785,0.846255
2,CatBoost,0.827154,0.817054,0.811448,0.821561,0.813708
3,LGBM,0.804757,0.804744,0.803633,0.805868,0.804757
4,SVM,0.786729,0.786729,0.786729,0.786729,0.786729
5,Logistic Regression,0.793521,0.801348,0.802484,0.803608,0.793508
6,kNN,0.810337,0.817104,0.821586,0.819388,0.827216


In [21]:
cv_acc1

[0.842896379525593,
 0.842896379525593,
 0.8215355805243446,
 0.8047565543071162,
 0.786729088639201,
 0.7991011235955057,
 0.7969288389513108]